In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

: 

In [ ]:
# Load AIS data
df = pd.read_csv("AIS_2024_12_31.csv")


In [ ]:
df.head(5)

In [ ]:
df.columns


In [ ]:
df.shape

In [7]:
# Remove duplicates based on MMSI and timestamp (BaseDateTime)
df = df.drop_duplicates(subset=["MMSI", "BaseDateTime"])


In [ ]:
df.shape

In [ ]:
df.head(5)

In [10]:
df['StatusBinary'] = df['Status'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
df.head(5)

In [12]:
import pandas as pd

# Example mapping function
def map_vessel_type(vessel_type):
    try:
        vessel_type = int(vessel_type)
        first_digit = int(str(vessel_type)[0])
    except (ValueError, TypeError):
        return 'Unknown'

    mapping = {
        1: 'Reserved',
        2: 'Wing In Ground',
        3: 'Special Category',
        4: 'High-Speed Craft',
        5: 'Special Category',
        6: 'Passenger',
        7: 'Cargo',
        8: 'Tanker',
        9: 'Other'
    }
    return mapping.get(first_digit, 'Unknown')

# Example: Apply on a DataFrame column
df['VesselTypeString'] = df['VesselType'].apply(map_vessel_type)




In [ ]:
df.head(5)

In [14]:
# Simple label encoding: A = 1, B = 2
transceiver_mapping = {'A': "Large Class", 'B': "Small Class"}
df['TransceiverEncoded'] = df['TransceiverClass'].map(transceiver_mapping)

In [15]:
df['BaseDateTime'] = pd.to_datetime(df['BaseDateTime'])


In [ ]:
df.head(5)

In [ ]:
columns_needed = [
    'MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading', 
    'VesselName', 'IMO', 'CallSign', 'VesselTypeString', 'StatusBinary',
    'Length', 'Width', 'Draft', 'TransceiverEncoded'
]

df_new = df[columns_needed]

# --- 5. Save to new CSV ---
df_new.to_csv("CLEAN_AIS_2024_12_31.csv", index=False)
print("New dataset saved as 'updated_ais_dataset.csv'")

In [21]:
df['heading_vs_cog_diff'] = abs(df['Heading'] - df['COG'])

In [ ]:
# Function to calculate loiter time per group (MMSI)
def loitering_time(group):
    # Calculate the difference in LAT and LON for consecutive rows
    group['LAT_diff'] = group['LAT'].diff().abs()
    group['LON_diff'] = group['LON'].diff().abs()
    
    # Mark as loitering if the movement is small (threshold < 0.0001)
    group['is_loitering'] = ((group['LAT_diff'] < 0.0001) & (group['LON_diff'] < 0.0001)).astype(int)
    
    # Return the sum of loitering flags per group
    return group['is_loitering']

# Apply the function to the grouped DataFrame
df['loiter_time'] = df.groupby('MMSI').apply(loitering_time).reset_index(drop=True)

# Now, you can check the result
print(df.head())


In [ ]:
# Apply the function to the grouped DataFrame
df['loiter_time'] = df.groupby('MMSI').apply(loitering_time).reset_index(drop=True)

# Generate a new CSV file with updated attributes
df.to_csv('updated_ships_data_with_loitering.csv', index=False)

# Confirm the action
print("✅ New CSV file 'updated_ships_data_with_loitering.csv' has been created with all attributes and new features.")

In [ ]:
df.head(20)